In [9]:
import time 
import cv2
import os
"""
to install it on conda => https://anaconda.org/conda-forge/python-dotenv
"""




import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
import sklearn
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
import cv2
import random
import os
from PIL import Image, ImageEnhance
from PIL import Image, ImageEnhance
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.decomposition import PCA
import pickle
import pandas as pd

from skimage import feature, exposure



In [10]:

def HistogramofOrientedGradients(image):
    resized_img = resize(image, (64, 128))
    fd, hog_image = feature.hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
    
    return (np.array(fd)).flatten()

N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25

def get_contour_pixels(bw_image):
    contours, _= cv2.findContours(
        bw_image, cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_NONE
        ) 
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
    
    img2 = bw_image.copy()[:,:,np.newaxis]
    img2 = np.concatenate([img2, img2, img2], axis = 2)
    
    # if self.show_images:
    #     for cnt in contours : 
    #         cv2.drawContours(img2, [cnt], 0, (255, 0, 0), 1)  
            
    #     plt.imshow(img2, cmap='gray')
    return contours

def get_hinge_features(bw_image):
    # if self.is_binary:
    #     bw_image, _ = self.preprocess_binary_image(img_file, self.sharpness_factor, self.bordersize)
    # else:
    #     bw_image, _ = self.preprocess_image(img_file, self.sharpness_factor, self.bordersize)
    
    contours = get_contour_pixels(bw_image)
    
    hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
        
    # print([len(cnt) for cnt in contours])
    for cnt in contours:
        n_pixels = len(cnt)
        if n_pixels <= LEG_LENGTH:
            continue
        
        points = np.array([point[0] for point in cnt])
        xs, ys = points[:, 0], points[:, 1]
        point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        x1s, y1s = point_1s[:, 0], point_1s[:, 1]
        x2s, y2s = point_2s[:, 0], point_2s[:, 1]
        
        phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
        phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
        
        indices = np.where(phi_2s > phi_1s)[0]
        
        for i in indices:
            phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
            phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
            hist[phi1, phi2] += 1
            
    normalised_hist = hist / np.sum(hist)
    feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
    
    return feature_vector


#Initialization:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
K_S = np.arange(3, 8)
def get_cold_features(bw_image, approx_poly_factor = 0.01 , sharpness_factor = 10 , bordersize = 3):

    #bw_image, _ = preprocess_image(img_file, sharpness_factor, bordersize)
    contours = get_contour_pixels(bw_image)
    
    rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
    feature_vectors = np.zeros((len(K_S), N_BINS))
    
    for j, k in enumerate(K_S):
        hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
        for cnt in contours:
            epsilon = approx_poly_factor * cv2.arcLength(cnt,True)
            cnt = cv2.approxPolyDP(cnt,epsilon,True)
            n_pixels = len(cnt)
            
            point_1s = np.array([point[0] for point in cnt])
            x1s, y1s = point_1s[:, 0], point_1s[:, 1]
            point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
            x2s, y2s = point_2s[:, 0], point_2s[:, 1]
            
            thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
            rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
            if(rhos.all()):
                rhos_log_space = np.log10(rhos)
            else:
                rhos_log_space = rhos
            
            quantized_rhos = np.zeros(rhos.shape, dtype=int)
            for i in range(N_RHO_BINS):
                quantized_rhos += (rhos_log_space < rho_bins_edges[i])
                
            for i, r_bin in enumerate(quantized_rhos):
                theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
                hist[r_bin - 1, theta_bin] += 1
            
        normalised_hist = hist / hist.sum()
        feature_vectors[j] = normalised_hist.flatten()
        
    return feature_vectors.flatten()

def get_hinge_with_cold(bw_image):
    hinge=get_hinge_features(bw_image)
    cold=get_cold_features(bw_image)
    return np.concatenate([hinge, cold])
    

In [11]:

from dotenv import load_dotenv

load_dotenv()

# input files directories
TEST_DIRECTORY = os.environ.get("TEST_DIRECTORY")

OUTPUT_DIRECTORY = os.environ.get("OUTPUT_DIRECTORY") 

predicted_class = []
time_list = []
images = []
# read images 
male_data=np.load('male_data_small.npy',allow_pickle=True)
female_data = np.load('female_data_small.npy',allow_pickle=True)



# for filename in os.listdir(TEST_DIRECTORY):
#     image = cv2.imread(os.path.join(TEST_DIRECTORY,filename), 0)
#     if image is not None:
#         start_time = time.time()

#         k=10
#         wshape=(100*k,142*k)

#         image=cv2.resize(image,wshape)
#         image = cv2.GaussianBlur(image,(3,3),cv2.BORDER_DEFAULT)

#         image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#                 cv2.THRESH_BINARY,11,2) # making the image binary
#         image = cv2.medianBlur(image, 5)
#         image = cv2.medianBlur(image, 5)
#         image = cv2.erode(image, None, iterations = 1)
#         images.append(image)

#         # predicted_class.append() # here call preprocessing function and the model
#         end_time = time.time() - start_time
#         if(end_time < .001):
#             time_list.append(.001)
#         else:
#             time_list.append(end_time)
#images=np.array(images)
images=np.concatenate((male_data,female_data))
cold_f=[]
hinge_f=[]
hog_f=[]
hinge_with_cold_f=[]


for i,img in enumerate(images):
    cold_f.append(get_cold_features(img))
    hinge_f.append(get_hinge_features(img))
    hog_f.append(HistogramofOrientedGradients(img))
    hinge_with_cold_f.append(get_hinge_with_cold(img))

cold_f=np.array(cold_f)
hinge_f=np.array(hinge_f)
hog_f=np.array(hog_f)
hinge_with_cold_f=np.array(hinge_with_cold_f)

#load models

cold_model=pickle.load(open('cold','rb'))
pca_cold=pickle.load(open('cold_pca','rb'))
cold_f=pca_cold.transform(cold_f)
y_cold_pred=cold_model.predict_proba(cold_f)
#############################################
hinge_model=pickle.load(open('hinge','rb'))
y_hinge_pred=hinge_model.predict_proba(hinge_f)
###############################################
hog_model=pickle.load(open('hog','rb'))
y_hog_pred=hog_model.predict_proba(hog_f)
#################################################
hinge_with_cold_model=pickle.load(open('hinge_with_cold','rb'))
pca_hinge_with_cold=pickle.load(open('hinge_with_cold_pca','rb'))
hinge_with_cold_f=pca_hinge_with_cold.transform(hinge_with_cold_f)
y_hinge_with_cold_pred=hinge_with_cold_model.predict_proba(hinge_with_cold_f)
##end of prediction 


y_pred=np.argmax(y_cold_pred+y_hinge_pred+y_hog_pred+y_hinge_with_cold_pred, axis=1)
print(y_pred)
y_test=np.concatenate((np.ones(male_data.shape[0]),
                np.zeros(female_data.shape[0])),
                axis=0)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


    


# result_file = open(OUTPUT_DIRECTORY + '/results.txt', 'w')
# time_file = open(OUTPUT_DIRECTORY + '/times.txt', 'w')
# for i in range(len(time_list)):
#     result_file.write(predicted_class[i] + '\n')
#     time_file.write(time[i] + '\n')

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 1.0
